# Web scraping

Este notebook é uma demonstração de como extrair dados de votação de algum evento parlamentar da Câmara dos Deputados.

Você pode utulizá-lo para extrair dados das votações que seguem o mesmo padrão html da utilizada nesse exemplo.

### Documentação das bibliotecas utilizadas: 

<a href="https://docs.python-requests.org/en/latest/" title="Requests">Requests</a>

<a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc/#" title="BeautifulSoup">BeautifulSoup</a>

<a href="https://pandas.pydata.org/docs/" title="Pandas">Pandas</a>

<a href="https://docs.python.org/3/library/re.html" title="Regular Expressions">Regular Expressions</a>

### Permissões:

Caso use esse algoritmo para raspagem, não utilizem urls que violem o <a href="https://www.camara.leg.br/robots.txt" title="Protocolo de Exclusão de Robôs">Protocolo de Exclusão de Robôs</a>:

- User-Agent: * 
- Disallow: /sileg/prop_lista* 
- Disallow: /internet/sileg/prop_lista* 
- Disallow: /sileg/Prop_lista* 
- Disallow: /internet/sileg/Prop_lista* 
- Disallow: /*arvore-de-apensados 
- Disallow: /proposicoesWeb/prop_arvore_tramitacoes 
- Disallow: /sileg/prop_arvore_tramitacoes 
- Disallow: /internet/deputado/Dep_Lista* 
- Disallow: /transparencia/recursos-humanos/remuneracao/*
- Disallow: /transparencia/recursos-humanos/contratos-terceirizacao* 
- Disallow: /transparencia/recursos-humanos/funcionarios* 

In [1]:
# importando as bibliotecas necessárias
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re # expressões regulares

In [2]:
def extract_html(url):
    """Extract the html code of the page.
    
    Args:
        url (str): page's url

    Returns:
        text (str): html extracted
    """
    
    try:
        r = requests.get(url) 
        text = r.text 
    except:
        raise HTTPException(404)
        
    return text


Abaixo temos um exemplo de um elemento da lista que contém todas as informações que precisamos por palarmentar.

    <li>
      <span class="nome">Abou Anni</span>
      <span class="nomePartido">(PSL-SP)</span>
      <span class="votou">-votou<span class="voto sim">Sim</span></span>
     </li>

Precisamos navegar até o elemento de cada deputado e extrair as informações relevantes.

In [3]:
def scrape_page(text):
    """Extract from the input 
    
    :param text: recebe o html como um objeto string.
    
    :returns array: array com os dados raspados de shape (número de votantes, número de informações).
    
    """
    # crinado o objeto soup
    soup = BeautifulSoup(text) 

    # visualizando o objeto soup:
    # print(soup.prettify()) # com identação
    
    # inicializa a lista
    array = []
    
    # percorre o item de lista com a info:
    for elemento in soup.find_all('span', attrs={"class":"nome"}):
        lista = []
        lista.append(elemento.parent.contents[1].string) # nome do parlamentar
        
        lista.append(elemento.parent.contents[3].string) # partido e estado
        
        
        # Voto
        if not elemento.parent.select('span.votou span'): # lista vazia não tem método .string:
            lista.append('Sem voto') # ausência
        else:
            lista.append(elemento.parent.select('span.votou span')[0].string) # voto
        array.append(lista)
    return array
        
    

In [4]:
def save_data(array, nome='saved'):
    """
    Converte a array para dataframe e salva num arquivo .csv.
    
    Argumentos:
    array -- objeto que armazena os dados
    nome -- nome do arquivo a ser criado
    """
    df = pd.DataFrame(array, columns=['nome', 'partido', 'voto'])
    nome = nome + '.csv'
    df.to_csv(nome, index=False)
    return
    

In [7]:
def scraper(url, titulo):
    html = extract_html(url)
    array = scrape_page(html)
    save_data(array, titulo)
    return

In [8]:
if __name__ == '__main__':
    url = 'https://www.camara.leg.br/presenca-comissoes/votacao-portal?reuniao=62583&itemVotacao=9968&ordenacao=Nome'
    scraper(url, 'pec135')

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="app-id=1049343067" name="apple-itunes-app"/>
  <meta content="app-id=br.leg.camara.infolegmovel" name="google-play-app"/>
  <title>
   Portal da Câmara dos Deputados
  </title>
  <style type="text/css">
   @import url(https://www2.camara.leg.br/portal_css/camara_leg_viradadigital/member-cachekey-01563218998c9e36fc22b9a9e975ff88.css);
  </style>
  <style type="text/css">
   @import url(https://www2.camara.leg.br/portal_css/camara_leg_viradadigital/portlets-cachekey-996c2c14a4a7f928d077328f2e83bf9d.css);
  </style>
  <style media="screen" type="text/css">
   @import url(https://www2.camara.leg.br/portal_css/camara_leg_viradadigital/deprecated-cachekey-bd45b4a65d3ba1a4c60a6c49fd20abab.css);
  </style>
  <style media="screen" type="text/css">
   @import url(https://www2.camara.leg.